In [0]:
pip install zingg

Processing ./zingg-0.5.0-py2.py3-none-any.whl
  Using cached py4j-0.10.9-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached py4j-0.10.9-py2.py3-none-any.whl (198 kB)
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
pip install zingg

In [0]:
pip show zingg

Name: zingg
Version: 0.5.0
Summary: Zingg Entity Resolution, Data Mastering and Deduplication
Home-page: https://github.com/zinggAI/zingg
Author: Zingg.AI
Author-email: sonalgoyal4@gmail.com
License: https://github.com/zinggAI/zingg/blob/main/LICENSE
Location: /local_disk0/.ephemeral_nfs/envs/pythonEnv-2d07ea11-92af-46c0-89b5-f43fcae067a6/lib/python3.12/site-packages
Requires: py4j
Required-by: 


In [0]:
dbutils.library.restartPython()

In [0]:
!pip install tabulate

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
zinggDir = "/models"
modelId = "zinggTestApr1"

In [0]:
MARKED_DIR = zinggDir + "/" + modelId + "/trainingData/marked/"
UNMARKED_DIR = zinggDir + "/" + modelId + "/trainingData/unmarked/"
MARKED_DIR_DBFS = "/dbfs" + MARKED_DIR
UNMARKED_DIR_DBFS = "/dbfs" + UNMARKED_DIR

import pandas as pd
import numpy as np
 
import time
import uuid
 
from tabulate import tabulate
from ipywidgets import widgets, interact, GridspecLayout
import base64
import pyspark.sql.functions as fn

##this code sets up the Zingg Python interface
from zingg.client import *
from zingg.pipes import *

def cleanModel():
    dbutils.fs.rm(MARKED_DIR, recurse=True)
    # drop unmarked data
    dbutils.fs.rm(UNMARKED_DIR, recurse=True)
    return

# assign label to candidate pair
def assign_label(candidate_pairs_pd, z_cluster, label):
  '''
  The purpose of this function is to assign a label to a candidate pair
  identified by its z_cluster value.  Valid labels include:
     0 - not matched
     1 - matched
     2 - uncertain
  '''
  
  # assign label
  candidate_pairs_pd.loc[ candidate_pairs_pd['z_cluster']==z_cluster, 'z_isMatch'] = label
  
  return
 
def count_labeled_pairs(marked_pd):
  '''
  The purpose of this function is to count the labeled pairs in the marked folder.
  '''
  n_total = len(np.unique(marked_pd['z_cluster']))
  n_positive = len(np.unique(marked_pd[marked_pd['z_isMatch']==1]['z_cluster']))
  n_negative = len(np.unique(marked_pd[marked_pd['z_isMatch']==0]['z_cluster']))
  
  return n_positive, n_negative, n_total
# setup widget 
available_labels = {
    'No Match':0,
    'Match':1,
    'Uncertain':2
    }

/databricks/spark/python/pyspark/sql/context.py:117: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [0]:
#build the arguments for zingg
args = Arguments()
# Set the modelid and the zingg dir. You can use this as is
args.setModelId(modelId)
args.setZinggDir(zinggDir)

In [0]:
schema = "rec_id string, fname string, lname string, stNo string, add1 string, add2 string, city string, state string, dob string, ssn string"
inputPipe = CsvPipe("testFebrl", "/FileStore/tables/test.csv", schema)
args.setData(inputPipe)

set schema 


In [0]:
#setting outputpipe in 'args'
outputPipe = CsvPipe("resultOutput", "/tmp/outputApr1")
args.setOutput(outputPipe)

In [0]:
rec_id = FieldDefinition("rec_id", "string", MatchType.DONT_USE)
fname = FieldDefinition("fname", "string", MatchType.FUZZY)  # First Name
lname = FieldDefinition("lname", "string", MatchType.FUZZY)  # Last Name
stNo = FieldDefinition("stNo", "string", MatchType.FUZZY)    # Street Number
add1 = FieldDefinition("add1", "string", MatchType.FUZZY)    # Address Line 1
add2 = FieldDefinition("add2", "string", MatchType.FUZZY)    # Address Line 2
city = FieldDefinition("city", "string", MatchType.FUZZY)    # City
state = FieldDefinition("state", "string", MatchType.FUZZY)  # State
dob = FieldDefinition("dob", "string", MatchType.EXACT)      # Date of Birth (prefer exact match)
ssn = FieldDefinition("ssn", "string", MatchType.EXACT)      # SSN (should use exact match)


In [0]:
# Create the field definitions list
fieldDefs = [rec_id, fname, lname, stNo, add1, add2, city, state, dob, ssn]
# Set field definitions in args
args.setFieldDefinition(fieldDefs)

In [0]:
args.setNumPartitions(4)
args.setLabelDataSampleSize(0.5)

In [0]:
options = ClientOptions([ClientOptions.PHASE,"findTrainingData"])
#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()

['--phase', 'findTrainingData']
arguments for client options are  ['--phase', 'findTrainingData', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']


In [0]:
options = ClientOptions([ClientOptions.PHASE,"label"])
#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.init()

['--phase', 'label']
arguments for client options are  ['--phase', 'label', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']


In [0]:
# get candidate pairs
candidate_pairs_pd = getPandasDfFromDs(zingg.getUnmarkedRecords())
 
# if no candidate pairs, run job and wait
if candidate_pairs_pd.shape[0] == 0:
  print('No unlabeled candidate pairs found.  Run findTraining job ...')
else:
    # get list of pairs (as identified by z_cluster) to label 
    z_clusters = list(np.unique(candidate_pairs_pd['z_cluster'])) 
    # identify last reviewed cluster
    last_z_cluster = '' # none yet
    # print candidate pair stats
    print('{0} candidate pairs found for labeling'.format(len(z_clusters)))

/databricks/spark/python/pyspark/sql/dataframe.py:162: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


21 candidate pairs found for labeling


In [0]:
# Label Training Set
# define variable to avoid duplicate saves
ready_for_save = False
print(candidate_pairs_pd)
# user-friendly labels and corresponding zingg numerical value
# (the order in the dictionary affects how displayed below)
LABELS = {
  'Uncertain':2,
  'Match':1,
  'No Match':0  
  }
# GET CANDIDATE PAIRS
# ========================================================
#candidate_pairs_pd = get_candidate_pairs()
n_pairs = int(candidate_pairs_pd.shape[0]/2)
# ========================================================
# DEFINE IPYWIDGET DISPLAY
# ========================================================
display_pd = candidate_pairs_pd.drop(
  labels=[
    'z_zid', 'z_prediction', 'z_score', 'z_isMatch', 'z_zsource'
    ], 
  axis=1)
# define header to be used with each displayed pair
html_prefix = "<p><span style='font-family:Courier New,Courier,monospace'>"
html_suffix = "</p></span>"
header = widgets.HTML(value=f"{html_prefix}<b>" + "<br />".join([str(i)+"&nbsp;&nbsp;" for i in display_pd.columns.to_list()]) + f"</b>{html_suffix}")
# initialize display
vContainers = []
vContainers.append(widgets.HTML(value=f'<h2>Indicate if each of the {n_pairs} record pairs is a match or not</h2></p>'))
# for each set of pairs
for n in range(n_pairs):
  # get candidate records
  candidate_left = display_pd.loc[2*n].to_list()
  print(candidate_left)
  candidate_right = display_pd.loc[(2*n)+1].to_list()
  print(candidate_right)
  # define grid to hold values
  html = ''
  for i in range(display_pd.shape[1]):
    # get column name
    column_name = display_pd.columns[i]
    # if field is image
    if column_name == 'image_path':
      # define row header
      html += '<tr>'
      html += '<td><b>image</b></td>'
      # read left image to encoded string
      l_endcode = ''
      if candidate_left[i] != '':
        with open(candidate_left[i], "rb") as l_file:
          l_encode = base64.b64encode( l_file.read() ).decode()
      # read right image to encoded string
      r_encode = ''
      if candidate_right[i] != '':
        with open(candidate_right[i], "rb") as r_file:
          r_encode = base64.b64encode( r_file.read() ).decode()      
      # present images
      html += f'<td><img src="data:image/png;base64,{l_encode}"></td>'
      html += f'<td><img src="data:image/png;base64,{r_encode}"></td>'
      html += '</tr>'
    elif column_name != 'image_path':  # display text values
      if column_name == 'z_cluster': z_cluster = candidate_left[i]
      html += '<tr>'
      html += f'<td style="width:10%"><b>{column_name}</b></td>'
      html += f'<td style="width:45%">{str(candidate_left[i])}</td>'
      html += f'<td style="width:45%">{str(candidate_right[i])}</td>'
      html += '</tr>'
  # insert data table
  table = widgets.HTML(value=f'<table data-title="{z_cluster}" style="width:100%;border-collapse:collapse" border="1">'+html+'</table>')
  z_cluster = None
  # assign label options to pair
  label = widgets.ToggleButtons(
    options=LABELS.keys(), 
    button_style='info'
    )
  # define blank line between displayed pair and next
  blankLine=widgets.HTML(value='<br>')
  # append pair, label and blank line to widget structure
  vContainers.append(widgets.VBox(children=[table, label, blankLine]))
# present widget
display(widgets.VBox(children=vContainers))
# ========================================================
# mark flag to allow save 
ready_for_save = True

    z_zid         z_cluster  z_prediction  ...   dob       ssn  z_zsource
0      43  1743509306782:10          -1.0  ...   vic  19510806  testFebrl
1      20  1743509306782:10          -1.0  ...   vic  19461101  testFebrl
2      22  1743509306782:11          -1.0  ...   vic  19461101  testFebrl
3      21  1743509306782:11          -1.0  ...   vic  19461101  testFebrl
4      46  1743509306782:14          -1.0  ...    sa  19250817  testFebrl
5      43  1743509306782:14          -1.0  ...   vic  19510806  testFebrl
6      31  1743509306782:15          -1.0  ...   vic  19191031  testFebrl
7      21  1743509306782:15          -1.0  ...   vic  19461101  testFebrl
8      57  1743509306782:19          -1.0  ...   qld  19801125  testFebrl
9      31  1743509306782:19          -1.0  ...   vic  19191031  testFebrl
10      4   1743509306782:2          -1.0  ...    ss            testFebrl
11      3   1743509306782:2          -1.0  ...    sa  19830807  testFebrl
12     63  1743509306782:23          -

In [0]:
if not ready_for_save:
  print('No labels have been assigned. Run the previous cell to create candidate pairs and assign labels to them before re-running this cell.')

else:

  # ASSIGN LABEL VALUE TO CANDIDATE PAIRS IN DATAFRAME
  # ========================================================
  # for each pair in displayed widget
  for pair in vContainers[1:]:

    # get pair and assigned label
    html_content = pair.children[1].get_interact_value() # the displayed pair as html
    user_assigned_label = pair.children[1].get_interact_value() # the assigned label

    # extract candidate pair id from html pair content
    start = pair.children[0].value.find('data-title="')
    if start > 0: 
      start += len('data-title="') 
      end = pair.children[0].value.find('"', start+2)
    pair_id = pair.children[0].value[start:end]



    # assign label to candidate pair entry in dataframe
    candidate_pairs_pd.loc[candidate_pairs_pd['z_cluster']==pair_id, 'z_isMatch'] = LABELS.get(user_assigned_label)
  # ========================================================

  # SAVE LABELED DATA TO ZINGG FOLDER
  # ========================================================
  # make target directory if needed
  dbutils.fs.mkdirs(MARKED_DIR)
  
  # save label assignments
  # save labels
  zingg.writeLabelledOutputFromPandas(candidate_pairs_pd,args)

  # count labels accumulated
  marked_pd_df = getPandasDfFromDs(zingg.getMarkedRecords())
  n_pos, n_neg, n_tot = count_labeled_pairs(marked_pd_df)
  print(f'You have accumulated {n_pos} pairs labeled as positive matches.')
  print(f'You have accumulated {n_neg} pairs labeled as not matches.')
  print("If you need more pairs to label, re-run the cell for 'findTrainingData'")
  # ========================================================  

  # save completed
  ready_for_save = False

/databricks/spark/python/pyspark/sql/dataframe.py:162: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


You have accumulated 8 pairs labeled as positive matches.
You have accumulated 13 pairs labeled as not matches.
If you need more pairs to label, re-run the cell for 'findTrainingData'


In [0]:
options = ClientOptions([ClientOptions.PHASE,"updateLabel"])
markedRecords = getPandasDfFromDs(zingg.getMarkedRecords())

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.init()
print(markedRecords)

['--phase', 'updateLabel']
arguments for client options are  ['--phase', 'updateLabel', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']
    z_zid         z_cluster  z_prediction  ...   dob       ssn  z_zsource
0       4   1743509306782:2          -1.0  ...    ss            testFebrl
1       3   1743509306782:2          -1.0  ...    sa  19830807  testFebrl
2      63  1743509306782:23          -1.0  ...    wa  19030926  testFebrl
3      61  1743509306782:23          -1.0  ...    wa  19030926  testFebrl
4      62  1743509306782:27          -1.0  ...    wa  19030926  testFebrl
5      61  1743509306782:27          -1.0  ...    wa  19030926  testFebrl
6      57   1743509306782:3          -1.0  ...   qld  19801125  testFebrl
7      21   1743509306782:3          -1.0  ...   vic  19461101  testFebrl
8      41  1743509306782:31          -1.0  ...   vic  19510806  testFebrl
9      16  1743509306782:31          -1.0  ...   qld  19180205  testFebrl
10     25  1

/databricks/spark/python/pyspark/sql/dataframe.py:162: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [0]:
options = ClientOptions([ClientOptions.PHASE,"trainMatch"])
# Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()

['--phase', 'trainMatch']
arguments for client options are  ['--phase', 'trainMatch', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']


In [0]:
outputDF1 = spark.read.csv("/tmp/outputApr1")
colNames = ["z_minScore", "z_maxScore", "z_cluster", "rec_id", "fname", "lname", "stNo", "add1", "add2", "city", "state", "dob", "ssn"]
outputDF1.toDF(*colNames).show(100)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8148892167456287>, line 1
----> 1 outputDF1 = spark.read.csv("/tmp/outputApr1")
      2 colNames = ["z_minScore", "z_maxScore", "z_cluster", "rec_id", "fname", "lname", "stNo", "add1", "add2", "city", "state", "dob", "ssn"]
      3 outputDF1.toDF(*colNames).show(100)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:837, in DataFrameReader.csv(self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ign

In [0]:
options = ClientOptions([ClientOptions.PHASE,"generateDocs"])
#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()
DOCS_DIR = zinggDir + "/" + modelId + "/docs/"
dbutils.fs.ls('file:'+DOCS_DIR)

# see the labels
displayHTML(open(DOCS_DIR+"model.html", 'r').read())

['--phase', 'generateDocs']
arguments for client options are  ['--phase', 'generateDocs', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']


Zingg Model Documentation 
 
 
 
 
 
 
 
 Model zinggTestApr1 
 
 
 
 
 
 Unmarked 0/21, Marked 21/21 ( 8 Matches , 13 Non-Matches , 0 Unsure) 
 
 
 
 
 

 
 
 
 CLUSTER 
 
 Z_SCORE 
 Z_ISMATCH 
 REC_ID 
 FNAME 
 LNAME 
 STNO 
 ADD1 
 ADD2 
 CITY 
 STATE 
 DOB 
 SSN 
 Z_ZSOURCE 
 
 
 

 

 
 
 1743509306782:10
 
 
 
 
0

 
 
 
0

 
 
 
rec-1036-org

 
 
 
 amber

 
 
 
 held

 
 
 
24

 
 
 
 lampard circuit

 
 
 
 emerald garden

 
 
 
 golden bay

 
 
 
2447

 
 
 
 vic

 
 
 
19510806

 
 
 
testFebrl

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
rec-1029-dup-2

 
 
 
 annalise

 
 
 
 stephenson

 
 
 
81

 
 
 
 rose scott circuit

 
 
 
 cordoba manor

 
 
 
 ashfoeld

 
 
 
4226

 
 
 
 vic

 
 
 
19461101

 
 
 
testFebrl

 
 
 
 
 
 
 1743509306782:11
 
 
 
 
0

 
 
 
1

 
 
 
rec-1029-dup-4

 
 
 
 kylee

 
 
 
 stephenson

 
 
 
81

 
 
 
 cordoba manor

 
 
 
 rose scott circuit

 
 
 
 ashfield

 
 
 
4226

 
 
 
 vic

 
 
 
19461101

 
 
 
testFebrl

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
rec-1029-dup-3

 
 
 
 kykee

 
 
 
 turale

 
 
 
81

 
 
 
 rose scott circuit

 
 
 
 

 
 
 
 ashfield

 
 
 
4226

 
 
 
 vic

 
 
 
19461101

 
 
 
testFebrl

 
 
 
 
 
 
 1743509306782:14
 
 
 
 
0

 
 
 
0

 
 
 
rec-1039-dup-0

 
 
 
 angus

 
 
 
 roas

 
 
 
62

 
 
 
 gormansto crescent

 
 
 
 mlc centre

 
 
 
 kiruwah

 
 
 
3350

 
 
 
 sa

 
 
 
19250817

 
 
 
testFebrl

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
rec-1036-org

 
 
 
 amber

 
 
 
 held

 
 
 
24

 
 
 
 lampard circuit

 
 
 
 emerald garden

 
 
 
 golden bay

 
 
 
2447

 
 
 
 vic

 
 
 
19510806

 
 
 
testFebrl

 
 
 
 
 
 
 1743509306782:15
 
 
 
 
0

 
 
 
0

 
 
 
rec-1033-org

 
 
 
 keziah

 
 
 
 painter

 
 
 
18

 
 
 
 ainslie avenue

 
 
 
 sec 1

 
 
 
 torquay

 
 
 
3205

 
 
 
 vic

 
 
 
19191031

 
 
 
testFebrl

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
rec-1029-dup-3

 
 
 
 kykee

 
 
 
 turale

 
 
 
81

 
 
 
 rose scott circuit

 
 
 
 

 
 
 
 ashfield

 
 
 
4226

 
 
 
 vic

 
 
 
19461101

 
 
 
testFebrl

 
 
 
 
 
 
 1743509306782:19
 
 
 
 
0

 
 
 
0

 
 
 
rec-1042-dup-1

 
 
 
 kiandra

 
 
 
 cowle

 
 
 
2

 
 
 
 gatliff place

 
 
 
 rustenubr g sth

 
 
 
 girgarre

 
 
 
3995

 
 
 
 qld

 
 
 
19801125

 
 
 
testFebrl

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
rec-1033-org

 
 
 
 keziah

 
 
 
 painter

 
 
 
18

 
 
 
 ainslie avenue

 
 
 
 sec 1

 
 
 
 torquay

 
 
 
3205

 
 
 
 vic

 
 
 
19191031

 
 
 
testFebrl

 
 
 
 
 
 
 1743509306782:2
 
 
 
 
0

 
 
 
1

 
 
 
rec-1022-dup-2

 
 
 
 Érik

 
 
 
 Guay

 
 
 
840

 
 
 
 fowles street

 
 
 
 moun tvjiew

 
 
 
 burleigh heads

 
 
 
2830

 
 
 
 ss

 
 
 
 

 
 
 
testFebrl

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
rec-1022-dup-1

 
 
 
 Érik

 
 
 
 Guay

 
 
 
840

 
 
 
 mountview

 
 
 
 fowles treet

 
 
 
 burlei gh heads

 
 
 
2803

 
 
 
 sa

 
 
 
19830807

 
 
 
testFebrl

 
 
 
 
 
 
 1743509306782:23
 
 
 
 
0

 
 
 
1

 
 
 
rec-1044-dup-3

 
 
 
 nicole

 
 
 
 carbone

 
 
 
46

 
 
 
 schlich street

 
 
 
 simpsonary barracks

 
 
 
 toowoomba

 
 
 
3000

 
 
 
 wa

 
 
 
19030926

 
 
 
testFebrl

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
rec-1044-dup-1

 
 
 
 nicole

 
 
 
 carbone

 
 
 
46

 
 
 
 schlich nstreet

 
 
 
 simpson army barracks

 
 
 
 toowoomba

 
 
 
3000

 
 
 
 wa

 
 
 
19030926

 
 
 
testFebrl

 
 
 
 
 
 
 1743509306782:27
 
 
 
 
0

 
 
 
1

 
 
 
rec-1044-dup-2

 
 
 
 nicole

 
 
 
 carbone

 
 
 
46

 
 
 
 schlich street

 
 
 
 simpson arm ybarracks

 
 
 
 toowong

 
 
 
3000

 
 
 
 wa

 
 
 
19030926

 
 
 
testFebrl

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
rec-1044-dup-1

 
 
 
 nicole

 
 
 
 carbone

 
 
 
46

 
 
 
 schlich nstreet

 
 
 
 simpson army barracks

 
 
 
 toowoomba

 
 
 
3000

 
 
 
 wa

 
 
 
19030926

 
 
 
testFebrl

 
 
 
 
 
 
 1743509306782:3
 
 
 
 
0

 
 
 
0

 
 
 
rec-1042-dup-1

 
 
 
 kiandra

 
 
 
 cowle

 
 
 
2

 
 
 
 gatliff place

 
 
 
 rustenubr g sth

 
 
 
 girgarre

 
 
 
3995

 
 
 
 qld

 
 
 
19801125

 
 
 
testFebrl
